<a href="https://colab.research.google.com/github/zhouy185/exercise-medical-cost-revisited/blob/main/Exercise_Medical_Cost_Prediction_Revisited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise: Medical cost prediction revisited

Insurance companies need to predict the annual medical cost of a insurance policy holder.

* Target: **annual medical cost**
* Predictors:
    * age, gender, bmi, number of children, smoker/non-smoke, region

## Data Preparation

Let's load the [dataset](https://raw.githubusercontent.com/zhouy185/BUS_O712/refs/heads/main/Data/medical_costs.csv).

In [1]:
import pandas as pd
df = pd.read_csv("medical_costs.csv")
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830
1333,18,female,31.920,0,no,northeast,2205.98080
1334,18,female,36.850,0,no,southeast,1629.83350
1335,21,female,25.800,0,no,southwest,2007.94500


We will now seperate the data into X and y, and then perform the train-test split.

In [2]:
# target vs. features
y = df['charges']
X = df.drop('charges',axis=1)

# Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

Next, construct a column transformer that performs one-hot-encoding

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Define the categorical and numerical features

cat_cols = ['sex', 'smoker', 'region']
num_cols = ['age', 'bmi', 'children']

# Define the column transformer

ct = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(sparse_output=False, drop="first"), cat_cols),
    ],
    remainder="passthrough"  # Pass through numerical columns without scaling
)

# Linear Regression

Let's define a Linear Regression model and include it in a pipeline.

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# Predictive model
lr = LinearRegression()


# Define the pipeline
pipe = Pipeline(
    steps=[
        ("ct", ct),
        ("lr", lr),
    ]
)


Fit the pipeline.

In [5]:
pipe.fit(X_train,y_train)

,steps,"[('ct', ...), ('lr', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


Predict on the test data and calculate R2 and RMSE

In [8]:
y_pred = pipe.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
import math

mse = mean_squared_error(y_test,y_pred)
rmse_lr = math.sqrt(mse)
r2_lr = r2_score(y_test,y_pred)
print(rmse_lr,r2_lr)


5956.342894363594 0.8069287081198009


# Random Forest Regressor

We now use Random Forest to perform the regression.

In [9]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()


Include the model in the pipeline, and fit the pipeline

In [10]:
pipe = Pipeline(
    steps=[
        ("ct", ct),
        ("rf", rf),
    ]
)

pipe.fit(X_train,y_train)

,steps,"[('ct', ...), ('rf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


Now predict on test data and evaluate the performance.

In [12]:
y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test,y_pred)
rmse_rf = math.sqrt(mse)

r2_rf = r2_score(y_test,y_pred)

print(rmse_rf,r2_rf)

4720.691662608587 0.8787254728723515


# XGBoost for Regression

Define the XGBoost Model

In [22]:
from xgboost import XGBRegressor

xgb = XGBRegressor(
    n_estimators=100,
    learning_rate=0.01,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)


ImportError: cannot import name 'PANDAS_INSTALLED' from 'xgboost.compat' (/Users/matthewzychowicz/Desktop/MBA/O712_DataAnalPy/W11/exercise-medical-cost-revisited-Matt-Zychowicz/.venv/lib/python3.12/site-packages/xgboost/compat.py)

Include the model in a pipeline.

In [16]:
pipe = Pipeline(
    steps=[
        ("ct", ct),
        ("xgb", xgb),
    ]
)

pipe.fit(X_train,y_train)

NameError: name 'xgb' is not defined

Performance evaluation

In [17]:
y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test,y_pred)
rmse_xgb = math.sqrt(mse)

r2_xgb = r2_score(y_test,y_pred)

print(rmse_xgb,r2_xgb)

4720.691662608587 0.8787254728723515


# LightGBM

Define a LGBM mode.

In [19]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(
    n_estimators=100,
    max_depth=10,
)

OSError: dlopen(/Users/matthewzychowicz/Desktop/MBA/O712_DataAnalPy/W11/exercise-medical-cost-revisited-Matt-Zychowicz/.venv/lib/python3.12/site-packages/lightgbm/lib/lib_lightgbm.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib
  Referenced from: <D44045CD-B874-3A27-9A61-F131D99AACE4> /Users/matthewzychowicz/Desktop/MBA/O712_DataAnalPy/W11/exercise-medical-cost-revisited-Matt-Zychowicz/.venv/lib/python3.12/site-packages/lightgbm/lib/lib_lightgbm.dylib
  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Vol

Including the model in a pipeline.

In [20]:
pipe = Pipeline(
    steps=[
        ("ct", ct),
        ("lgbm", lgbm),
    ]
)

pipe.fit(X_train,y_train)

NameError: name 'lgbm' is not defined

Performance evaluation

In [21]:
y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test,y_pred)
rmse_lgb = math.sqrt(mse)

r2_lgb = r2_score(y_test,y_pred)

print(rmse_lgb,r2_lgb)

4720.691662608587 0.8787254728723515


## KNN for Regression

We need the `KNeighborsRegressor()` function from `sklearn.neighbors` to create a KNN model for regression. Recall that we need to specify the number of neighbors.

We will define the KNN model and included in a pipe line

In [23]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=20)

But can we use the same column transformer? Why?

In [24]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler


ct_1he_scale = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(), cat_cols)
    ],
    remainder="passthrough"  # Pass through numerical columns without scaling
  )

Combine the column transformer and the knn model in a pipeline.

In [25]:
pipe = Pipeline(
    steps=[
        ("ct", ct_1he_scale),
        ("knn", knn),
    ]
)

pipe.fit(X_train,y_train)

,steps,"[('ct', ...), ('knn', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


Let's predict on the test data and again compute the RMSE and $R^2$ score.

In [26]:
y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test,y_pred)
rmse = math.sqrt(mse)

r2 = r2_score(y_test,y_pred)

print(rmse,r2)

12266.026105644482 0.18122214246654367


Let's perform a grid search with a 5-fold cross-validation for the KNN regression model.


In [29]:
from sklearn.model_selection import GridSearchCV, KFold

param_grid ={
    'knn__n_neighbors': [10, 20, 30, 40, 50]
}

kf = KFold()

grid_search = GridSearchCV(
    estimator = pipe,
    param_grid = param_grid,
    scoring = 'r2',
    n_jobs = -1,
    cv = kf
)

grid_search.fit(X_train,y_train)


,estimator,Pipeline(step...ighbors=20))])
,param_grid,"{'knn__n_neighbors': [10, 20, ...]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split...shuffle=False)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('cat', ...)]"


Let's retrieve the best KNN regression model by cross-validation, and predict on the test data set.

In [31]:
knn_best = grid_search.best_estimator_

y_pred = knn_best.predict(X_test)

mse = mean_squared_error(y_test,y_pred)
rmse = math.sqrt(mse)
r2 = r2_score(y_test,y_pred)
print(rmse,r2)

12145.039150929188 0.19729464849327594


In [32]:
grid_search.best_params_

{'knn__n_neighbors': 10}

## Among all models we used, which one has the best performance?

Provide your answer below.

LightGBM